# Common Codes

In [1]:
# Importing all the required libraries
import datetime
import time
import os
import json
# import sys
import pandas as pd
# import numpy as np
import logging
import requests
import traceback

import uuid
from dataclasses import dataclass, field
import yfinance as yahoo_finance
import random

import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from email.mime.base import MIMEBase
from email import encoders 

In [2]:
# Global variables or constants
INPUT_PATH = "/data/workspace_files/input/"
OUTPUT_PATH = "/data/workspace_files/output/"

MAIL_SERVER = "aaa.com"
MAIL_PORT = 0
SENDER_ID = "xyz@em.com"
RECEIVER_IDS = ("abc@em.com", "")

In [3]:
SIMPLE_EMAIL_MESSAGE = """
<html>
    <h1>Test mail</h1>
    <p>This is a Test mail</p>
</html>
"""

In [3]:
# configuring the logger to print logs
log_format1 = '%(asctime)s [%(levelname)-8s] <PID %(process)d:%(processName)s> %(name)s.%(funcName)s: %(message)s'
log_format2 = '%(asctime)s [%(levelname)-8s] %(name)s.%(funcName)s: %(message)s'
log_format3 = '%(asctime)s [%(levelname)-8s] [%(processName)s] %(name)s: %(message)s'

formatter = logging.Formatter(log_format3, datefmt='%d-%b-%Y %H:%M:%S')

console = logging.StreamHandler()
console.setFormatter(formatter)

file_handler = logging.FileHandler(OUTPUT_PATH + '/logs/stdout.log', "a")
file_handler.setFormatter(formatter)

logger = logging.getLogger("Python Exercises")

logger.handlers.clear()

logger.addHandler(console)
logger.addHandler(file_handler)

logger.setLevel(logging.INFO)

### Generic Functions

In [5]:
def get_file_paths(src, extension=None, mindepth=1, maxdepth=float('inf')):
    """

Arguments:

Returns:
    """
    rootdir = os.path.normcase(src)
    file_paths = []
    root_depth = rootdir.rstrip(os.path.sep).count(os.path.sep) - 1
    for dirpath, dirs, files in os.walk(rootdir):
        depth = dirpath.count(os.path.sep) - root_depth
        if mindepth <= depth <= maxdepth:
            for filename in files:
                if extension is None:
                    file_paths.append(os.path.join(dirpath, filename))
                if os.path.splitext(filename)[1] == extension:
                    file_paths.append(os.path.join(dirpath, filename))
        elif depth > maxdepth:
            del dirs[:]
    return file_paths

In [6]:
def get_file_names(paths):
    """

Arguments:

Returns:
    """
    urls = []
    if type(paths) is str:
        urls.append(paths)
    if type(paths) is list:
        urls = paths

    file_names = [url.split(os.path.sep)[-1] for url in urls]

    return file_names

In [7]:
def download_file(src, dest):
    """
Function to dowload file(s) to a specified location
Returns a dictionary object stating details like source, destination, started date time, end date time and timetaken.
Arguments:
src - Location from which file needs to be downloaded
dest - Location at which file needs to be saved
Returns:
log - a dictionary object having all the details regarding the file download
    """
    log = {
        "source": src,
        "destination": dest,
        "started_on": datetime.datetime.now(),
        "completed_on": None,
        "time_taken": -1,
        "status": "Failed",
        "description": "UnKnown"
    }
    start = time.perf_counter()
    # noinspection PyBroadException
    try:
        response = requests.get(src)
        open(f"{dest}", "wb").write(response.content)
        log["status"] = "Successful".upper()
        log["description"] = "File is successfully dowloaded."
    except Exception as e:
        log["status"] = "Failed".upper()
        log["description"] = traceback.format_exc()

    end = time.perf_counter()
    log["time_taken"] = (end - start)
    log["completed_on"] = datetime.datetime.now()

    return log

In [8]:
def flatten_json(y):
    """

Arguments:

Returns:
    """
    out = {}

    def flatten(x, name=''):
        if type(x) is dict:
            for a in x:
                flatten(x[a], name + a + '_')
        elif type(x) is list:
            i = 0
            for a in x:
                flatten(a, name + str(i) + '_')
                i += 1
        else:
            out[name[:-1]] = x

    flatten(y)
    return out

In [9]:
def send_email(mail_from=SENDER_ID,
               mail_to=RECEIVER_IDS,
               subject=None,
               message_text=None,
               file=None
               ):
    mail_sent = False
    try:
        message = MIMEMultipart("alternative")
        message["Subject"] = subject
        message["From"] = mail_from
        message["To"] = ", ".join(mail_to)
        message.attach(MIMEText(message_text, 'html'))
        if file is not None:
            part = MIMEBase('application', "octet-stream")
            part.set_payload(open(file, "rb").read())
            encoders.encode_base64(part)
            _, tail = os.path.split(file)
            part.add_header('Content-Disposition',
                            'attachment; filename={}'.format(str(tail)))
            message.attach(part)
        print("Message is : \n", message.as_string())
        smtp = smtplib.SMTP(MAIL_SERVER, MAIL_PORT)
        smtp.sendmail(mail_from, mail_to, message.as_string())
        mail_sent = True
        smtp.quit()
    except Exception as e:
        print(e)
    return mail_sent

In [10]:
def profile_data(df:pd.DataFrame):
    """
Type inference: detect the types of columns in a data frame.
Essentials: type, unique values, missing values
Quantile statistics like minimum value, Q1, median, Q3, maximum, range, inter-quartile range
Descriptive statistics like mean, mode, standard deviation, sum, median absolute deviation, coefficient of variation, kurtosis, skewness
Most frequent values
    """
    column_name_final = ["column_name", "data_type", "count", "uniqueness", "missing", "min", "max", "q1", "q25", "q50",
                         "q75", "q95", "mean", "mode", "std_dev", "sum", "kurtosis", "skewness", "top_5_values",
                         "most_freq_values"
                         ]
    print(column_name_final)
    pass

# Exercise 1

## Objective – Data Analysis


### Tasks
1. Diamond Prices 
   
   Diamonds are divided into five impurity types based on the structure of their carbon atoms. The [Diamonds dataset](https://www.kaggle.com/shivam2503/diamonds) from Kaggle gives you even more info — cut, clarity, color, and price. Develop your data visualization skills on it with some exploratory data analysis. 
2. Age of Abalone Shells
   
    This is a unique [dataset from zoology](https://www.kaggle.com/rodolfomendes/abalone-dataset). Abalone shells are miracles of nature, and you can determine their age by counting the circles inside their shells. Can you determine the age of Abalone shells with Python data analysis skills?


### Inputs

### Outputs

### Task 1: – Diamond Data

In [ ]:
def diamond_data_exploration():
    diamond_df = pd.read_csv(INPUT_PATH + "diamonds.csv")
    diamond_df = diamond_df.rename(columns={"Unnamed: 0": "id"})
    return diamond_df

In [12]:
diamond_df.dtypes

In [23]:
diamond_df

In [24]:
diamond_df.describe()

In [38]:
diamond_df.groupby(by=["cut"]).count().reset_index()

In [37]:
diamond_df.groupby(by=["clarity"]).count().reset_index()

In [36]:
diamond_df.groupby(by=["color"]).count().reset_index()

### Task 2: – Abalone Age data

In [ ]:
abalone_df = pd.read_csv(INPUT_PATH + "abalone.csv")

In [ ]:
abalone_df.dtypes

In [ ]:
abalone_df

In [ ]:
abalone_df.groupby("Rings").count().reset_index()

# Exercise 2

## Objective – Web Scraping


### Tasks


### Inputs

### Outputs

In [27]:
from typing import List, Optional

In [11]:
@dataclass
class Links:
    type: Optional[str]
    url: str
    language: Optional[str]

In [17]:
@dataclass
class Chapter:
    number: int
    title: str
    status: str
    released_on: datetime.datetime
    modified_on: datetime.datetime
    links: List[Links]

In [18]:
@dataclass
class Episode:
    number: int
    title: str
    description: str
    status: str
    released_on: datetime.datetime
    modified_on: datetime.datetime
    links: List[Links]

In [19]:
@dataclass
class Season:
    number: int
    title: str
    description: str
    status: str
    modified_on: datetime.datetime
    episodes: List[Episode]

In [20]:
@dataclass
class Volume:
    number: int
    title: str
    status: str
    modified_on: datetime.datetime
    chapters: List[Chapter]

In [21]:
@dataclass
class Video:
    number: int
    title: str
    status: str
    modified_on: datetime.datetime
    links: List[Season]

In [28]:
@dataclass
class Catalog:
    name: Optional[str]
    description: Optional[str]
    country_of_origin: Optional[str]
    genres: Optional[List[str]]
    tags: Optional[List[str]]
    other_names: Optional[List[str]]
    links: Optional[List[Links]]
    novels: Optional[List[Volume]]
    manga: Optional[List[Volume]]
    anime: Optional[List[Video]]
    id: str = field(default_factory = lambda: str(uuid.uuid4()))
    released_on: datetime.datetime = field(default_factory = lambda: datetime.datetime.now())
    created_on: datetime.datetime= field(default_factory = lambda : datetime.datetime.now())

In [ ]:
catalog_json = {
    "id": str(uuid.uuid4()),
    "name": None,
    "description": "",
    
}

In [ ]:
def build_catalog_data():
    pass

# Exercise 3

## Objective – Stock Price Prediction


### Tasks


### Inputs

### Outputs

In [11]:
def get_symbols_from_csv():
    symobls_df = pd.read_csv(INPUT_PATH + "nasdaq_symbols_data.csv")
    symobls_df = symobls_df.astype({"Symbol": str, "Name": str})
    symbols = symobls_df['Symbol']
    return symbols

In [12]:
# noinspection PyBroadException
def get_stock_data(symbols):
    # download or scrape data from the url or API or webpage
    stock_info = {}
    stock_data = None
    failed_symbols = []
    file_name = "stocks_data_{index}.csv"
    file_index = 1
    file_path = OUTPUT_PATH + file_name
    for index, symbol in enumerate(symbols):
        try:
            ticker = yahoo_finance.Ticker(symbol)
            stock_info[symbol] = ticker.info
            # Let us  get historical stock prices for specified Symbol or Ticker covering the past few years.
            # max-> maximum number of daily prices available
            # Valid options are 1d, 5d, 1mo, 3mo, 6mo, 1y, 2y, 5y, 10y and ytd.
            ticker_history = ticker.history(period="max")
            ticker_history.insert(0, "symbol", symbol)
            if stock_data is None:
                stock_data = ticker_history.copy(deep=True)
                ticker_history.to_csv(file_path.format(index = file_index), header = True, mode='w', index = True, index_label = "date_time")
            else:
                #stock_data = pd.concat([stock_data, ticker_history], ignore_index=False)
                ticker_history.to_csv(file_path.format(index = file_index), header = False, mode='a', index = True, index_label = "date_time")

            logger.info({"ticker_index":index, "ticker": symbol, "status": "SUCCESSFUL", "description": "Data written to file " + file_name.format(index = file_index)})
            
            if os.path.getsize(file_path.format(index = file_index)) >= 246415360: # 268435456 to 246415360 i.e. 256MiB to 235MiB
                file_index += 1
                stock_data = None
        except Exception:
            logger.error({"ticker_index":index, "ticker": symbol, "status": "FAILED", "description": traceback.format_exc()})
            failed_symbols.append(symbol)
        #time.sleep(1)
    with open(OUTPUT_PATH + "stock_info.json", "w", encoding='utf-8') as f:
        json.dump(stock_info, f)
    logger.info("List of failed symbols are : "+ str(failed_symbols))
    return stock_data

In [ ]:
def train_stock_prediction_model():
    pass

In [13]:
tickers = get_symbols_from_csv()
logger.info("Total Symbols present are: " + str(len(tickers)))
get_stock_data(tickers)
#stock_data_df = get_stock_data(tickers)
#stock_data_df.to_csv(OUTPUT_PATH + "stocks_data_1.csv", mode='w', index = True, index_label = "date_time")

06-Sep-2023 10:32:55 [INFO    ] [MainProcess] Python Exercises: {'ticker': 'GTY', 'status': 'SUCCESSFUL', 'description': 'Data written to file stocks_data_5.csv'}
06-Sep-2023 10:32:58 [INFO    ] [MainProcess] Python Exercises: {'ticker': 'GUG', 'status': 'SUCCESSFUL', 'description': 'Data written to file stocks_data_5.csv'}
06-Sep-2023 10:33:02 [INFO    ] [MainProcess] Python Exercises: {'ticker': 'GURE', 'status': 'SUCCESSFUL', 'description': 'Data written to file stocks_data_5.csv'}
06-Sep-2023 10:33:06 [INFO    ] [MainProcess] Python Exercises: {'ticker': 'GUT', 'status': 'SUCCESSFUL', 'description': 'Data written to file stocks_data_5.csv'}
06-Sep-2023 10:33:09 [INFO    ] [MainProcess] Python Exercises: {'ticker': 'GUT^C', 'status': 'FAILED', 'description': 'Traceback (most recent call last):\n  File "<ipython-input-12-1beaad720e94>", line 13, in get_stock_data\n    stock_info[symbol] = ticker.info\n  File "/opt/python/envs/default/lib/python3.8/site-packages/yfinance/ticker.py", l

,symbol,Open,High,Low,Close,Volume,Dividends,Stock Splits
Date,,,,,,,,
2021-10-29 00:00:00-04:00,UDMY,27.000000,27.740000,26.010000,27.500000,7601900,0.0,0.0
2021-11-01 00:00:00-04:00,UDMY,28.000000,29.690001,26.110001,29.219999,2189400,0.0,0.0
2021-11-02 00:00:00-04:00,UDMY,29.219999,29.590000,27.705000,29.459999,1371000,0.0,0.0
2021-11-03 00:00:00-04:00,UDMY,29.590000,29.990000,27.129999,28.600000,1338900,0.0,0.0
2021-11-04 00:00:00-04:00,UDMY,28.980000,29.030001,27.219999,27.379999,2029700,0.0,0.0
...,...,...,...,...,...,...,...,...
2023-08-30 00:00:00-04:00,UDMY,10.150000,10.480000,10.145000,10.450000,390700,0.0,0.0
2023-08-31 00:00:00-04:00,UDMY,10.490000,10.670000,10.330000,10.360000,477400,0.0,0.0
2023-09-01 00:00:00-04:00,UDMY,10.500000,10.630000,10.205000,10.230000,434700,0.0,0.0


# Exercise 4

## Objective – Log file Parsing

### Tasks


### Inputs

### Outputs

In [13]:
json_data = []

In [24]:
with open(OUTPUT_PATH + "/logs/stdout_01.log", "r") as f:
    i = 0
    for line in f.readlines():
        if i==0:
            i += 1
            continue
        #print(line)
        #print(line[64:])
        if "SUCCESSFUL" in line:
            json_data.append(json.loads(line[64:].replace("'", '"')))

In [25]:
json_data

[{'ticker': 'A',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'A',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AA',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AAC',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AACG',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AACI',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AACIW',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AACT',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AADI',
  'status': 'SUCCESSFUL',
  'description': 'Data written to file stocks_data_1.csv'},
 {'ticker': 'AAIC',
  'status': 'SUCCESSFUL',
  'descr

In [27]:
len(json_data)

7115

In [28]:
sdf = pd.DataFrame(json_data)

In [30]:
sdf["file_name"] = sdf.description.str.extract('(stocks_data_[0-9]{1,2}.csv)')

In [52]:
result = sdf.groupby("file_name").apply(lambda x: set(x["ticker"].values.tolist())).reset_index(name='tickers_list')

In [53]:
result["sorted_tickers_list"] = result.tickers_list.sort_values().apply(lambda x: sorted(x))

In [54]:
result

,file_name,tickers_list,sorted_tickers_list
0,stocks_data_1.csv,"{AACT, ADVM, AMR, AIRS, ADOCW, ADEX, ALYA, API...","[A, AA, AAC, AACG, AACI, AACIW, AACT, AADI, AA..."
1,stocks_data_10.csv,"{TWOA, TNP, SRT, SRI, SND, SPIR, SOHU, UCL, SO...","[SMTC, SMTI, SMWB, SMX, SMXWW, SN, SNA, SNAL, ..."
2,stocks_data_11.csv,"{WTRG, VIPS, UUUU, YCBD, ZIONP, UTME, UZE, VOR...","[UDMY, UDR, UE, UEC, UEIC, UFAB, UFCS, UFI, UF..."
3,stocks_data_2.csv,"{CFFS, CAPL, BUJAW, CBRE, BANR, BGXX, BZ, BSX,...","[BACA, BACK, BAER, BAERW, BAFN, BAH, BAK, BALL..."
4,stocks_data_3.csv,"{CPSI, CYN, CMTG, CYT, CPHI, DISTW, CNSL, COO,...","[CIF, CIFR, CIFRW, CIG, CIGI, CII, CIK, CIM, C..."
5,stocks_data_4.csv,"{FHLT, ETWO, FERG, FPI, EXPE, ESQ, FIACW, ELDN...","[DXCM, DXF, DXLG, DXPE, DXR, DXYN, DY, DYAI, D..."
6,stocks_data_5.csv,"{HCDIZ, GDO, GILD, GLUE, HDSN, HROWM, IHIT, GS...","[GAB, GABC, GAIA, GAIN, GAINL, GAINN, GAINZ, G..."
7,stocks_data_6.csv,"{LOCL, LC, LSXMB, JMM, JT, JLS, JOE, IVCBW, LU...","[ILPT, IMAB, IMACW, IMAQ, IMAQR, IMAQW, IMAX, ..."
8,stocks_data_7.csv,"{NOA, MUJ, MSN, NFJ, NICK, MBI, MET, NKX, MTZ,...","[MASS, MAT, MATH, MATV, MATW, MATX, MAV, MAX, ..."
9,stocks_data_8.csv,"{PINE, PIRS, OMER, PCAR, PNM, OCG, OXLC, NVTA,...","[NRAC, NRACW, NRBO, NRC, NRDS, NRDY, NREF, NRG..."


In [55]:
result["ticker_list_len1"] = result["tickers_list"].apply(len)
result["ticker_list_len2"] = result["sorted_tickers_list"].apply(len)

In [56]:
result

,file_name,tickers_list,sorted_tickers_list,ticker_list_len1,ticker_list_len2
0,stocks_data_1.csv,"{AACT, ADVM, AMR, AIRS, ADOCW, ADEX, ALYA, API...","[A, AA, AAC, AACG, AACI, AACIW, AACT, AADI, AA...",706,706
1,stocks_data_10.csv,"{TWOA, TNP, SRT, SRI, SND, SPIR, SOHU, UCL, SO...","[SMTC, SMTI, SMWB, SMX, SMXWW, SN, SNA, SNAL, ...",615,615
2,stocks_data_11.csv,"{WTRG, VIPS, UUUU, YCBD, ZIONP, UTME, UZE, VOR...","[UDMY, UDR, UE, UEC, UEIC, UFAB, UFCS, UFI, UF...",585,585
3,stocks_data_2.csv,"{CFFS, CAPL, BUJAW, CBRE, BANR, BGXX, BZ, BSX,...","[BACA, BACK, BAER, BAERW, BAFN, BAH, BAK, BALL...",677,677
4,stocks_data_3.csv,"{CPSI, CYN, CMTG, CYT, CPHI, DISTW, CNSL, COO,...","[CIF, CIFR, CIFRW, CIG, CIGI, CII, CIK, CIM, C...",624,624
5,stocks_data_4.csv,"{FHLT, ETWO, FERG, FPI, EXPE, ESQ, FIACW, ELDN...","[DXCM, DXF, DXLG, DXPE, DXR, DXYN, DY, DYAI, D...",619,619
6,stocks_data_5.csv,"{HCDIZ, GDO, GILD, GLUE, HDSN, HROWM, IHIT, GS...","[GAB, GABC, GAIA, GAIN, GAINL, GAINN, GAINZ, G...",653,653
7,stocks_data_6.csv,"{LOCL, LC, LSXMB, JMM, JT, JLS, JOE, IVCBW, LU...","[ILPT, IMAB, IMACW, IMAQ, IMAQR, IMAQW, IMAX, ...",692,692
8,stocks_data_7.csv,"{NOA, MUJ, MSN, NFJ, NICK, MBI, MET, NKX, MTZ,...","[MASS, MAT, MATH, MATV, MATW, MATX, MAV, MAX, ...",575,575
9,stocks_data_8.csv,"{PINE, PIRS, OMER, PCAR, PNM, OCG, OXLC, NVTA,...","[NRAC, NRACW, NRBO, NRC, NRDS, NRDY, NREF, NRG...",639,639


In [57]:
result['ticker_list_str'] = [','.join(i) if isinstance(i, list) else i for i in result['sorted_tickers_list']]

In [59]:
result[["file_name", "ticker_list_str"]]

,file_name,ticker_list_str
0,stocks_data_1.csv,"A,AA,AAC,AACG,AACI,AACIW,AACT,AADI,AAIC,AAIN,A..."
1,stocks_data_10.csv,"SMTC,SMTI,SMWB,SMX,SMXWW,SN,SNA,SNAL,SNAP,SNAX..."
2,stocks_data_11.csv,"UDMY,UDR,UE,UEC,UEIC,UFAB,UFCS,UFI,UFPI,UFPT,U..."
3,stocks_data_2.csv,"BACA,BACK,BAER,BAERW,BAFN,BAH,BAK,BALL,BALY,BA..."
4,stocks_data_3.csv,"CIF,CIFR,CIFRW,CIG,CIGI,CII,CIK,CIM,CINF,CING,..."
5,stocks_data_4.csv,"DXCM,DXF,DXLG,DXPE,DXR,DXYN,DY,DYAI,DYN,DYNT,D..."
6,stocks_data_5.csv,"GAB,GABC,GAIA,GAIN,GAINL,GAINN,GAINZ,GALT,GAM,..."
7,stocks_data_6.csv,"ILPT,IMAB,IMACW,IMAQ,IMAQR,IMAQW,IMAX,IMCC,IMC..."
8,stocks_data_7.csv,"MASS,MAT,MATH,MATV,MATW,MATX,MAV,MAX,MAXN,MAYS..."
9,stocks_data_8.csv,"NRAC,NRACW,NRBO,NRC,NRDS,NRDY,NREF,NRG,NRGV,NR..."


In [61]:
result[["file_name", "ticker_list_str"]].to_html(index=False)

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th>file_name</th>\n      <th>ticker_list_str</th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <td>stocks_data_1.csv</td>\n      <td>A,AA,AAC,AACG,AACI,AACIW,AACT,AADI,AAIC,AAIN,AAL,AAMC,AAME,AAN,AAOI,AAON,AAP,AAPL,AAT,AAU,AB,ABBV,ABCB,ABCL,ABCM,ABEO,ABEV,ABG,ABIO,ABL,ABLLW,ABLV,ABLVW,ABM,ABNB,ABOS,ABR,ABSI,ABT,ABUS,ABVC,AC,ACA,ACAB,ACABW,ACACU,ACACW,ACAD,ACAQ,ACAX,ACAXR,ACAXU,ACAXW,ACB,ACBA,ACBAW,ACCD,ACCO,ACDC,ACDCW,ACEL,ACER,ACET,ACGL,ACGLN,ACGLO,ACHC,ACHL,ACHR,ACHV,ACI,ACIC,ACIU,ACIW,ACLS,ACLX,ACM,ACMR,ACN,ACNB,ACNT,ACON,ACONW,ACOR,ACP,ACR,ACRE,ACRO,ACRS,ACRV,ACRX,ACST,ACT,ACTG,ACU,ACV,ACVA,ACXP,ADAG,ADAP,ADBE,ADC,ADCT,ADD,ADEA,ADER,ADERW,ADES,ADEX,ADI,ADIL,ADM,ADMA,ADMP,ADN,ADNT,ADNWW,ADOC,ADOCR,ADOCW,ADP,ADPT,ADRT,ADSE,ADSEW,ADSK,ADT,ADTH,ADTHW,ADTN,ADTX,ADUS,ADV,ADVM,ADVWW,ADX,ADXN,AE,AEAE,AEAEW,AEE,AEF,AEFC,AEG,AEHL,AEHR,AEI,AEIS,AEL,AEM,AEMD,AENT,AENZ,AEO,AEON,AEP,AER,AES,AESC,A

# Exercise T

## Objective – A

### Tasks


### Inputs

### Outputs

# References

- https://www.datacamp.com/blog/60-python-projects-for-all-levels-expertise